# Ch. 5. Distributed Algorithms

## I. Cluster Architecture

### Example - 5.1 Connecting to and Requesting Data From All Hosts

## II. Messages Between Systems

### Sidenote on MPI

### Example 5.2 - Parsl IPyParallel Tasks

## III. Distributed Dataflows

### Example 5.3 - Doing Distributed Math With Parsl

## IV. Large Scale Problems

### Example 5.4 - Basic Physics Dataflow

## Exercise 5. Real Live Workflow!

In [1]:
# Your code goes here